# MarkovChain DataFrames
Examples.

### State Space - String. Target Space - String (words)
The state space logically is lists of character of length n
where n is the order of the Markov chain.

In [1]:
import numpy as np
import pandas as pd
import sys
import argparse
import json
import random
from datetime import date, timedelta, time
from music21 import note, interval, duration
np.random.seed(42)

In [2]:
#
# input string is 'ABACACBABA', order is 2
# A user constraint is to identify the initial token (start of the word)
# with a space character. So in this example, ' A'
# Other user contraints - 
# 1. case sensitivity (or not)
# 2. indentify the end of a word with a special token.
#    This needs to be a character not found in the sample set, in this case use a trailing space
# 3. maximum/minimum output length
# 4. user specify initial seed OR auto select (initial or any)
# The constraints are configurable.
# 
my_index = [' A','AB','AC','BA','CA','CB','A ']
my_columns = ['A','B','C',' ']
my_data = [[0, 1.0, 0,0],[1.0,0,0,0],[0.5,0.5,0,0],[0, 0.3333, 0.3333,0.3333],[0,0,1.0,0],[1.0,0,0,0],[0,0,0,0]]


In [3]:
my_data

[[0, 1.0, 0, 0],
 [1.0, 0, 0, 0],
 [0.5, 0.5, 0, 0],
 [0, 0.3333, 0.3333, 0.3333],
 [0, 0, 1.0, 0],
 [1.0, 0, 0, 0],
 [0, 0, 0, 0]]

In [4]:
chain = pd.DataFrame(data=my_data,index=my_index,columns=my_columns)

In [5]:
chain

,A,B,C,
A,0.0,1.0000,0.0000,0.0000
AB,1.0,0.0000,0.0000,0.0000
AC,0.5,0.5000,0.0000,0.0000
BA,0.0,0.3333,0.3333,0.3333
CA,0.0,0.0000,1.0000,0.0000
CB,1.0,0.0000,0.0000,0.0000
A,0.0,0.0000,0.0000,0.0000


In [6]:
# dict for each my_index entry
# consists of a list of counts for each column
# Probabilities calculated later from the counts
#
dict0 = {my_index[0]:[0, 1, 0, 0] }
dict1 = {my_index[1]:[1, 0, 0, 0] }
dict2 = {my_index[2]:[1, 1, 0, 0] }
dict3 = {my_index[3]:[0, 1, 1, 1] }
stats = [dict0,dict1,dict2,dict3]
stats

[{' A': [0, 1, 0, 0]},
 {'AB': [1, 0, 0, 0]},
 {'AC': [1, 1, 0, 0]},
 {'BA': [0, 1, 1, 1]}]

In [5]:
index_str = 'AB'
col_str = 'C'
df = pd.DataFrame(data=[1],index=[index_str], columns=[ col_str])
if len(df) == 0:
    df = pd.DataFrame(data=[1],index=[index_str], columns=[col_str])
df

,C
AB,1


In [6]:
df.index


Index(['AB'], dtype='object')

In [7]:
df

,C
AB,1


In [8]:
# new row, new column
df.loc['BC','D'] = 1
df = df.fillna(value=0)
df

,C,D
AB,1.0,0.0
BC,0.0,1.0


In [9]:
# existing row
if 'AB' in df.index:
    df.loc['AB','C'] = 1 + df.loc['AB','C']
df

,C,D
AB,2.0,0.0
BC,0.0,1.0


In [11]:
df.loc['BC','C'] = 1
df.loc['AB','D'] = 2
df.loc['AC','D'] = 4
df.fillna(0, inplace=True)
df

,C,D
AB,2.0,2.0
BC,1.0,1.0
AC,0.0,4.0


In [12]:
df = df.rename_axis('KEY')
df

,C,D
KEY,,
AB,2.0,2.0
BC,1.0,1.0
AC,0.0,4.0


In [13]:
# compute probabilities
sums = df.sum(axis=1)
chain = df.div(sums, axis=0)
chain

,C,D
KEY,,
AB,0.5,0.5
BC,0.5,0.5
AC,0.0,1.0


In [18]:
print(chain.sort_index(axis=0))
print(chain.sort_index(axis=1, ascending=False))

       C    D
KEY          
AB   0.5  0.5
AC   0.0  1.0
BC   0.5  0.5
       D    C
KEY          
AB   0.5  0.5
BC   0.5  0.5
AC   1.0  0.0


In [ ]:
#
# example WordProducer output, min length is 3. 
#
# acacba
# cacbabacba
# cacbababac
# babacacaca
# cbabacacac


### State Space - Intervals. Target Space - Measures
The state space is music21.interval.Interval of length n,
where n is the order of the Markov chain.

In [3]:
from music21 import stream, interval, corpus, note
#
# get intervals for a Part
#
def get_part_intervals(apart):
    intrvals = []
    part_notes = apart.flat.getElementsByClass('Note')
    for ind in range(len(part_notes)-1):
        n1 = part_notes[ind]
        n2 = part_notes[ind+1]
        i = interval.Interval(n1, n2)
        intrvals.append(i)
    return intrvals

def get_score_intervals(ascore):
    parts = ascore.getElementsByClass(stream.Part)
    pdict = dict()
    for p in parts:
        pname = p.partName
        intrvals = get_part_intervals(p)
        pdict[pname] = intrvals
    return pdict
    
def get_intervals_for_score(ascore):
    pdict = get_score_intervals(ascore)
    intrvals_df = pd.DataFrame()
    part_number = 1
    for k in pdict.keys():
            df = pd.DataFrame(data=pdict[k], columns=['interval'])
            df['part_number'] = part_number
            df['part_name'] = k
            intrvals_df = intrvals_df.append(df)
            part_number = part_number + 1

    intrvals_df['name'] = [x.name for x in intrvals_df['interval']]
    intrvals_df['niceName'] = [x.niceName for x in intrvals_df['interval']]
    intrvals_df['semitones'] = [x.semitones for x in intrvals_df['interval']]
    return intrvals_df
    

In [17]:
sBach = corpus.parse('bwv67.4')
print('len: {}  number of parts: {}'.format(len(sBach), len(sBach.parts)))
for sb in sBach.parts:
    print(sb, len(sb))
soprano = sBach.parts[0]
soprano_intervals = get_part_intervals(soprano)   # intervals for Soprano Part
print(len(soprano_intervals))

len: 6  number of parts: 4
<music21.stream.Part Soprano> 20
<music21.stream.Part Alto> 20
<music21.stream.Part Tenor> 20
<music21.stream.Part Bass> 20
38


In [18]:
s = soprano_intervals[0:10]
s
# for this example use the first 10 intervals as input
# to build an order-2 MarkovChain (as a DataFrame)
# 

[<music21.interval.Interval P1>,
 <music21.interval.Interval P1>,
 <music21.interval.Interval P5>,
 <music21.interval.Interval M2>,
 <music21.interval.Interval m2>,
 <music21.interval.Interval m-2>,
 <music21.interval.Interval M-2>,
 <music21.interval.Interval M-2>,
 <music21.interval.Interval M2>,
 <music21.interval.Interval M2>]

In [23]:
intervals_df = get_intervals_for_score(sBach)

In [25]:
for i in range(len(intervals_df)):
    print(intervals_df.iloc[i])

interval       <music21.interval.Interval P1>
part_number                                 1
part_name                             Soprano
name                                       P1
niceName                       Perfect Unison
semitones                                   0
Name: 0, dtype: object
interval       <music21.interval.Interval P1>
part_number                                 1
part_name                             Soprano
name                                       P1
niceName                       Perfect Unison
semitones                                   0
Name: 1, dtype: object
interval       <music21.interval.Interval P5>
part_number                                 1
part_name                             Soprano
name                                       P5
niceName                        Perfect Fifth
semitones                                   7
Name: 2, dtype: object
interval       <music21.interval.Interval M2>
part_number                                 1
part_name  

In [27]:
intervals = intervals_df.iloc[2:4]
print(intervals)

                         interval  part_number part_name name       niceName  \
2  <music21.interval.Interval P5>            1   Soprano   P5  Perfect Fifth   
3  <music21.interval.Interval M2>            1   Soprano   M2   Major Second   

   semitones  
2          7  
3          2  


In [28]:
# Use of initial token is optional, for this example it is omitted
# If used, a large interval not present in the Part would be used
# for example m59 or 99 (99 chromatic semitones)
# Also optional is a terminal Interval
#
zint = interval.Interval(99)
# 
# (P1,P1) --> P5
# (P1, P5) --> M2   etc.
#
# the columns are just the names (string)of unique intervals in the input
# directedName preserves the direction - up or down
my_columns = [interval.Interval('P1').directedName, interval.Interval('m2').directedName, 
              interval.Interval('M2').directedName, interval.Interval('P5').directedName, 
              interval.Interval('M-2').directedName, interval.Interval('m-2').directedName]
#
# the index is comprised of the directedName of interval pairs
my_index = []
for i in range(len(s)-1):
    t = '{},{}'.format(s[i].directedName,s[i+1].directedName)
    my_index.append(t)
my_index

['P1,P1',
 'P1,P5',
 'P5,M2',
 'M2,m2',
 'm2,m-2',
 'm-2,M-2',
 'M-2,M-2',
 'M-2,M2',
 'M2,M2']

In [57]:
my_columns

['P1', 'm2', 'M2', 'P5', 'M-2', 'm-2']

In [58]:
# initialize the probabilities to all 0.0
#
my_data = np.zeros((len(my_index),len(my_columns)))

In [67]:
mchain_df = pd.DataFrame(data=my_data, index=my_index, columns=my_columns)

In [68]:
mchain_df

,P1,m2,M2,P5,M-2,m-2
"P1,P1",0.0,0.0,0.0,0.0,0.0,0.0
"P1,P5",0.0,0.0,0.0,0.0,0.0,0.0
"P5,M2",0.0,0.0,0.0,0.0,0.0,0.0
"M2,m2",0.0,0.0,0.0,0.0,0.0,0.0
"m2,m-2",0.0,0.0,0.0,0.0,0.0,0.0
"m-2,M-2",0.0,0.0,0.0,0.0,0.0,0.0
"M-2,M-2",0.0,0.0,0.0,0.0,0.0,0.0
"M-2,M2",0.0,0.0,0.0,0.0,0.0,0.0
"M2,M2",0.0,0.0,0.0,0.0,0.0,0.0


In [72]:
mchain_df.loc['P1,P5']

P1     0.0
m2     0.0
M2     0.0
P5     0.0
M-2    0.0
m-2    0.0
Name: P1,P5, dtype: float64

In [77]:
mchain_df.to_json('mchain.json',orient="index" )
mchain_df.to_csv('mchain.csv')

In [81]:
# add the probabilities to the csv file and read it back in
# NOTE - had to delete the initial ',' in row 1
df = pd.read_csv('mchain.csv')
df

,P1,m2,M2,P5,M-2,m-2
"P1,P1",0.0,0.0,0.0,1.0,0.0,0.0
"P1,P5",0.0,0.0,1.0,0.0,0.0,0.0
"P5,M2",0.0,1.0,0.0,0.0,0.0,0.0
"M2,m2",0.0,0.0,0.0,0.0,0.0,1.0
"m2,m-2",0.0,0.0,0.0,0.0,1.0,0.0
"m-2,M-2",0.0,0.0,0.0,0.0,1.0,0.0
"M-2,M-2",0.0,0.0,1.0,0.0,0.0,0.0
"M-2,M2",0.0,0.0,1.0,0.0,0.0,0.0
"M2,M2",0.0,0.0,0.0,0.0,0.0,0.0


In [30]:
from pathlib import Path
def show_path_info(source):
    p = Path(source)
    print(f"source: {source} exists: {p.exists()}, is_dir: {p.is_dir()}, is_file: {p.is_file()}")
    print(f"parts: {p.parts}, name: {p.name}, suffix: {p.suffix} " )

source = "/Compile/music21/music21/corpus" 
show_path_info(source)

source = "/Compile/music21/music21/corpus/bach/bwv9.7.mxl"
show_path_info(source)

source: /Compile/music21/music21/corpus exists: True, is_dir: True, is_file: False
parts: ('\\', 'Compile', 'music21', 'music21', 'corpus'), name: corpus, suffix:  
source: /Compile/music21/music21/corpus/bach/bwv9.7.mxl exists: True, is_dir: False, is_file: True
parts: ('\\', 'Compile', 'music21', 'music21', 'corpus', 'bach', 'bwv9.7.mxl'), name: bwv9.7.mxl, suffix: .mxl 


In [16]:
parser = argparse.ArgumentParser()
parser.add_argument('infile', nargs='?', type=argparse.FileType('r'), default=sys.stdin)
parser.add_argument('outfile', nargs='?', type=argparse.FileType('w'),default=sys.stdout)
args = parser.parse_args(["resources/inlineTextChain.csv","out.txt"])
args

Namespace(infile=<_io.TextIOWrapper name='resources/inlineTextChain.csv' mode='r' encoding='cp1252'>, outfile=<_io.TextIOWrapper name='out.txt' mode='w' encoding='cp1252'>)

In [17]:
args.infile

<_io.TextIOWrapper name='resources/inlineTextChain.csv' mode='r' encoding='cp1252'>

In [18]:
for line in args.infile:
    #l = ' ' + line.strip() + '~'
    l = line.strip()
    print(l)

KEY,A,B,C,~
A,0.0,1.0,0.0,0.0
AB,1.0,0.0,0.0,0.0
AC,0.5,0.5,0.0,0.0
BA,0.0,0.3333333333333333,0.3333333333333333,0.3333333333333333
CA,0.0,0.0,1.0,0.0
CB,1.0,0.0,0.0,0.0


### WordProducer

In [6]:
order = 2
text_df = pd.read_json("resources/inlineText_charsChain.json", orient="index")
text_df_counts = pd.read_json("resources/inlineText_charCounts.json", orient="index")
text_df.rename_axis('KEY', inplace=True)
text_df

,A,B,C,D,R,~
KEY,,,,,,
A,0,1.000000,0.0,0.000000,0.00,0.000000
AB,0,0.000000,0.0,0.000000,1.00,0.000000
AC,1,0.000000,0.0,0.000000,0.00,0.000000
AD,1,0.000000,0.0,0.000000,0.00,0.000000
AR,0,1.000000,0.0,0.000000,0.00,0.000000
BA,0,0.333333,0.0,0.333333,0.00,0.333333
BR,1,0.000000,0.0,0.000000,0.00,0.000000
CA,0,0.000000,0.0,0.250000,0.75,0.000000
DA,0,0.500000,0.5,0.000000,0.00,0.000000


In [7]:
drugs_df = pd.read_json("resources/drugBrands_charsChain.json", orient="index")
drugs_df.rename_axis('KEY', inplace=True)
drugs_df.head()

,,-,1,2,3,a,b,c,d,e,...,r,s,t,u,v,w,x,y,z,~
KEY,,,,,,,,,,,,,,,,,,,,,
a,0.0,0.0,0.0,0.0,0.0,0.000000,0.03125,0.104167,0.125000,0.0000,...,0.104167,0.010417,0.0625,0.041667,0.062500,0.0,0.000000,0.020833,0.03125,0.0
b,0.0,0.0,0.0,0.0,0.0,0.140000,0.00000,0.020000,0.000000,0.4000,...,0.140000,0.000000,0.0000,0.020000,0.000000,0.0,0.000000,0.080000,0.00000,0.0
c,0.0,0.0,0.0,0.0,0.0,0.239583,0.00000,0.000000,0.000000,0.0625,...,0.062500,0.000000,0.0000,0.031250,0.000000,0.0,0.000000,0.125000,0.00000,0.0
d,0.0,0.0,0.0,0.0,0.0,0.160000,0.00000,0.000000,0.000000,0.3400,...,0.020000,0.000000,0.0000,0.220000,0.000000,0.0,0.000000,0.020000,0.00000,0.0
e,0.0,0.0,0.0,0.0,0.0,0.000000,0.00000,0.013699,0.041096,0.0000,...,0.068493,0.082192,0.0000,0.013699,0.054795,0.0,0.054795,0.013699,0.00000,0.0


In [17]:
s = drugs_df.to_csv()
# print(s)

In [11]:
# find an initial seed  
keys = pd.Series(drugs_df.index)
initial_keys = keys[keys.apply(lambda s:s.startswith(' '))]
initial_keys

0      a
1      b
2      c
3      d
4      e
5      f
6      g
7      h
8      i
9      j
10     k
11     l
12     m
13     n
14     o
15     p
16     q
17     r
18     s
19     t
20     u
21     v
22     w
23     x
24     y
25     z
Name: KEY, dtype: object

In [12]:
seed = initial_keys[random.randint(0, len(initial_keys)-1)]
seed

' r'

In [13]:
# any random key
seed = keys[random.randint(0, len(keys)-1)]
seed

'wn'

In [126]:
seed='in'
row = drugs_df.loc[seed]
row_probs = row[row > 0].cumsum()
# given a probability, pick the first character in the row_probs (cumulative probabilities)
# having a probability > the random probability
# for example:
prob = random.random()
print(row_probs)
print(prob)

-    0.013333
a    0.053333
d    0.066667
e    0.320000
i    0.346667
o    0.440000
t    0.480000
v    0.493333
z    0.506667
~    1.000000
Name: in, dtype: float64
0.5339347238087234


In [127]:
p = row_probs[row_probs> prob].iat[0]
nextc = row_probs[row_probs> prob].index[0]
print(f"random prob: {prob}, row prob: {p}, nextc: '{nextc}' ")


random prob: 0.5339347238087234, row prob: 0.9999999998, nextc: '~' 


In [128]:
next_seed = (seed + nextc)[1:order+1]
next_seed

'n~'

In [21]:
#
# breaks up a path name into component parts
#
import pathlib
def get_file_info(cpath, def_extension='json'):
    known_extensions = [def_extension, 'mxl','.xml','.musicxml']
    x = cpath.split("/")
    paths = x[0:len(x)-1]
    filename = x[-1]
    ext = filename.split(".")
    name = ext[0]
    if len(ext)==2 and ext[1] in known_extensions:
        ext = ext[1]
        path = cpath      
    else:
        ext = def_extension
        filename = f"{filename}.{ext}"
        path = f"{cpath}.{ext}"
    p = pathlib.Path(path)
    
    return {'paths':paths, 'path_text':path, 'filename':filename, 'name':name,'extension': ext, 'Path':p}

In [22]:
cpath = "resources/inlineTextChain"
file_info = get_file_info(cpath)
print(file_info)
print("file exists: {}".format(file_info['Path'].exists()))
print(str(file_info['Path']))

{'paths': ['resources'], 'path_text': 'resources/inlineTextChain.json', 'filename': 'inlineTextChain.json', 'name': 'inlineTextChain', 'extension': 'json', 'Path': WindowsPath('resources/inlineTextChain.json')}
file exists: True
resources\inlineTextChain.json


In [23]:
cpath = "/Compile/music21/music21/corpus/bach/bwv29.8"
file_info = get_file_info(cpath, def_extension='mxl')
print(file_info)
print("file exists: {}".format(file_info['Path'].exists()))
print(str(file_info['Path']))

{'paths': ['', 'Compile', 'music21', 'music21', 'corpus', 'bach'], 'path_text': '/Compile/music21/music21/corpus/bach/bwv29.8.mxl', 'filename': 'bwv29.8.mxl', 'name': 'bwv29', 'extension': 'mxl', 'Path': WindowsPath('/Compile/music21/music21/corpus/bach/bwv29.8.mxl')}
file exists: True
\Compile\music21\music21\corpus\bach\bwv29.8.mxl


In [20]:
cpath.split('.')

['/Compile/music21/music21/corpus/bach/bwv29', '8']

In [129]:
result = drugs_df.to_json(orient='index')
parsed = json.loads(result)
dumped = json.dumps(parsed, indent=4)
with open('temp.json', 'w') as f:
    f.write(str(dumped))

In [130]:
temp_df = pd.read_json('temp.json', orient="index")
temp_df

,,-,a,b,c,d,e,f,g,h,...,q,r,s,t,u,v,x,y,z,~
a,0,0.0,0.000000,0.038462,0.153846,0.115385,0.000000,0.0000,0.038462,0.000000,...,0.0,0.000000,0.000,0.115385,0.000000,0.076923,0.0000,0.000000,0.076923,0.00
b,0,0.0,0.058824,0.000000,0.000000,0.000000,0.470588,0.0000,0.000000,0.000000,...,0.0,0.058824,0.000,0.000000,0.235294,0.000000,0.0000,0.058824,0.000000,0.00
c,0,0.0,0.129032,0.000000,0.000000,0.000000,0.161290,0.0000,0.000000,0.032258,...,0.0,0.032258,0.000,0.000000,0.000000,0.000000,0.0000,0.064516,0.000000,0.00
d,0,0.0,0.000000,0.000000,0.000000,0.000000,0.230769,0.0000,0.000000,0.000000,...,0.0,0.000000,0.000,0.000000,0.230769,0.000000,0.0000,0.000000,0.000000,0.00
e,0,0.0,0.000000,0.000000,0.062500,0.125000,0.000000,0.0625,0.000000,0.000000,...,0.0,0.000000,0.125,0.000000,0.062500,0.000000,0.0625,0.000000,0.000000,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ze,0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,...,0.0,0.000000,0.200,0.200000,0.000000,0.000000,0.0000,0.000000,0.000000,0.00
zi,0,0.0,0.000000,0.000000,0.000000,0.200000,0.000000,0.0000,0.000000,0.000000,...,0.0,0.000000,0.000,0.200000,0.000000,0.000000,0.0000,0.000000,0.000000,0.20
zl,0,0.0,1.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,...,0.0,0.000000,0.000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.00
zo,0,0.0,0.000000,0.000000,0.000000,0.050000,0.000000,0.0000,0.000000,0.000000,...,0.0,0.100000,0.150,0.000000,0.000000,0.050000,0.0000,0.050000,0.000000,0.05


In [145]:
text_df.index

Index([' A', 'AB', 'AC', 'BA', 'CA', 'CB'], dtype='object', name='KEY')

In [146]:
text_df

,A,B,C,~
KEY,,,,
A,0.0,1.000000,0.000000,0.000000
AB,1.0,0.000000,0.000000,0.000000
AC,0.5,0.500000,0.000000,0.000000
BA,0.0,0.333333,0.333333,0.333333
CA,0.0,0.000000,1.000000,0.000000
CB,1.0,0.000000,0.000000,0.000000


In [7]:
#
# positional and keyword arguments
#
def my_func(*params1, **params2):
    print("params1: {} len: {} ".format(params1, len(params1)))
    print(params2)
    

In [8]:
my_func(1, 2, 3, sex=True, age=100)

params1: (1, 2, 3) len: 3 
{'sex': True, 'age': 100}


In [9]:
x = my_func   # function reference
x(4, 5, 6, name="Bill",age="66")

params1: (4, 5, 6) len: 3 
{'name': 'Bill', 'age': '66'}


In [8]:
s = "name=/data/corpus/x.mxl"

In [9]:
s.split('=')

['name', '/data/corpus/x.mxl']

In [21]:
durations_df = pd.read_csv('/Compile/dwbzen/resources/music/bwv29_8_durationsChain.csv')

In [15]:
durations_df['1.0'].apply(lambda x:round(x,3))

0     0.700
1     0.545
2     1.000
3     0.000
4     0.000
5     0.375
6     0.093
7     0.000
8     0.000
9     0.875
10    0.000
Name: 1.0, dtype: float64

In [27]:
def round_values(x, places=3):
    if not type(x) is str:
        return round(x, places)
    else:
        return x


In [30]:
durations_df = durations_df.applymap(lambda x:round_values(x,3))
durations_df

,KEY,1.0,2.0,0.5,1.5
0,"[1.0, 2.0]",0.700,0.100,0.200,0.000
1,"[1.0, 1.0]",0.545,0.455,0.000,0.000
2,"[2.0, 2.0]",1.000,0.000,0.000,0.000
3,"[1.0, 0.5]",0.000,0.000,1.000,0.000
4,"[0.5, 2.0]",0.000,0.875,0.125,0.000
5,"[2.0, 1.0]",0.375,0.375,0.188,0.062
6,"[0.5, 0.5]",0.093,0.070,0.837,0.000
7,"[2.0, 0.5]",0.000,0.000,1.000,0.000
8,"[1.5, 0.5]",0.000,0.000,1.000,0.000
9,"[0.5, 1.0]",0.875,0.125,0.000,0.000


## Interval chain

In [2]:
mc_intervalPath = "/Compile/dwbzen/resources/music/bwv29_8_intervalsChain.json"
durpath = "/Compile/dwbzen/resources/music/bwv29_8_durationsChain.json"
mc_countsPath = "/Compile/dwbzen/resources/music/bwv29_8_intervalCounts.json"
mc_interval_df = pd.read_json(mc_intervalPath, orient="index")
dur_df = pd.read_json(durpath, orient="index")
counts_df = pd.read_json(mc_countsPath, orient='index')
mc_interval_df.head()

,-5,1,-1,-2,2,5,-7,0,12,-3,4,7,-4,10,-6,3,100,9,-12
"[99, -1]",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0
"[-2, -5]",0.0,0.500000,0.000000,0.000000,0.500000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0
"[-5, 1]",0.0,0.000000,1.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0
"[1, -1]",0.0,0.166667,0.000000,0.750000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.083333,0.000000,0.0,0.000000,0.0,0.0,0.0
"[-1, -2]",0.0,0.000000,0.033333,0.566667,0.266667,0.033333,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.033333,0.0,0.066667,0.0,0.0,0.0


In [24]:
dur_df.columns.values
df_cols = dur_df.columns
cols = [float(str(x)[18:]) for x in dur_df.columns.values]
dur_df.rename({df_cols[0]:cols[0], df_cols[1]:cols[1],df_cols[2]:cols[2]},inplace=True)
dur_df


,1970-01-01 00:00:00.500,1970-01-01 00:00:02.000,1970-01-01 00:00:01.000
"[1.0, 1.0]",0.208,0.245,0.547
"[0.5, 0.5]",0.658,0.139,0.203
"[2.0, 1.0]",0.240,0.400,0.360
"[1.0, 0.5]",1.000,0.000,0.000
"[0.5, 2.0]",0.273,0.182,0.545
"[2.0, 2.0]",0.000,0.000,1.000
"[1.0, 2.0]",0.139,0.056,0.806
"[1.0, 1.5]",1.000,0.000,0.000
"[0.5, 1.0]",0.154,0.154,0.692
"[1.0, 3.0]",0.000,0.000,1.000


In [3]:
order = 2
keys =  pd.Series(mc_interval_df.index)
initial_keys = keys[keys.apply(lambda x: x.startswith('[99'))]
print(initial_keys)
key_values = keys.values
# pick any old random seed
seed = keys[random.randint(0, len(keys)-1)]
print('seed: {}'.format(seed))
print(keys.values)

0      [99, 0]
43    [99, -1]
dtype: object
seed: [-7, 12]
['[99, 0]' '[0, -2]' '[-2, -2]' '[-2, 2]' '[2, -2]' '[-2, -1]' '[-1, 1]'
 '[1, -1]' '[1, 0]' '[0, 0]' '[0, -1]' '[-1, -4]' '[-4, 5]' '[5, 5]'
 '[5, -6]' '[-6, 1]' '[-1, 0]' '[1, -3]' '[-3, 1]' '[1, 2]' '[2, -3]'
 '[2, 2]' '[-2, 9]' '[9, -2]' '[-2, 5]' '[5, 0]' '[-1, -2]' '[2, 1]'
 '[1, -5]' '[-5, 5]' '[5, -3]' '[-3, 0]' '[0, 5]' '[-4, 2]' '[0, -7]'
 '[-7, 9]' '[9, 0]' '[-1, 3]' '[3, -1]' '[5, -2]' '[-2, 0]' '[5, -1]'
 '[0, -5]' '[99, -1]' '[-2, -5]' '[-5, 1]' '[2, 5]' '[2, -7]' '[-7, 0]'
 '[0, 12]' '[12, -1]' '[1, -2]' '[-3, 2]' '[-7, 4]' '[4, 1]' '[-2, 7]'
 '[7, 1]' '[1, 1]' '[1, -7]' '[-7, 2]' '[2, 7]' '[7, -4]' '[-7, 12]'
 '[12, -2]' '[-1, 5]' '[-3, 5]' '[1, 4]' '[-7, 5]' '[-2, 10]' '[10, -2]'
 '[5, 7]' '[7, -6]' '[-6, -4]' '[1, -6]' '[-6, -2]' '[2, 3]' '[3, -2]'
 '[-2, -7]']


In [4]:
#
# pick next_token
#
row = mc_interval_df.loc[seed]
row_probs = row[row > 0].cumsum()
print('row_probs: {}'.format(row_probs))
prob = random.random()
p = row_probs[row_probs> prob].iat[0]
next_token = row_probs[row_probs> prob].index[0]
print('seed: {}  next token: {}'.format(seed, next_token))


row_probs: -2    0.5
-1    1.0
Name: [-7, 12], dtype: float64
seed: [-7, 12]  next token: -1


In [7]:
# determine new seed
ns = [int(x) for x in seed[1:len(seed)-1].split(',')]
ns.append(next_token)
print(ns)
new_seed = str(ns[1:])
print(new_seed)

[-7, 12, -1]
[12, -1]


## Notes chain

In [7]:
mc_notesPath = "/Compile/dwbzen/resources/music/bwv371_notesChain_dp.json"
durpath = "/Compile/dwbzen/resources/music/bwv371_durationsChain.json"
mc_notes_df = pd.read_json(mc_notesPath, orient="index")
dur_df = pd.read_json(durpath, orient="index")
mc_notes_df.head()

,C3,D3,G2,A2,B2,E3,F3,B-3,G3,A3,...,A4,D4,C#4,C4,G#4,E-4,D#4,B4,C5,D5
"C0,E3",1.0,0.000000,0.0,0.000000,0.000000,0.00,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"D3,C3",0.0,0.111111,0.0,0.222222,0.555556,0.00,0.0,0.0,0.111111,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"C3,D3",0.0,0.000000,0.4,0.000000,0.000000,0.50,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"D3,G2",0.2,0.400000,0.0,0.000000,0.000000,0.00,0.0,0.0,0.300000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"G2,D3",0.0,0.000000,0.0,0.250000,0.000000,0.75,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
keys =  pd.Series(mc_notes_df.index)
# print(keys)
initial_keys = keys[keys.apply(lambda x: x.startswith("['C0'"))]
print(initial_keys)
# pick any old random seed
seed = keys[random.randint(0, len(keys)-1)]
print('seed: {} {}'.format(seed, type(seed)))
#print(keys.values)

0      ['C0','E3']
89     ['C0','E4']
143    ['C0','B3']
170    ['C0','G4']
dtype: object
seed: ['C3','B2'] <class 'str'>


In [6]:
print("initial_keys: {} len: {}".format(initial_keys, len(initial_keys)))
ind = random.randint(0, len(initial_keys)-1)
print("index: {}".format(ind))
initial_keys.iloc[ind]

initial_keys: 0      ['C0','E3']
89     ['C0','E4']
143    ['C0','B3']
170    ['C0','G4']
dtype: object len: 4
index: 2


"['C0','B3']"

In [31]:
#
# pick next_token
#
row = mc_notes_df.loc[seed]
#print(row)
row_probs = row[row > 0].cumsum()
print('row_probs:\n{}'.format(row_probs))
prob = random.random()
p = row_probs[row_probs> prob].iat[0]
next_token = row_probs[row_probs> prob].index[0]
print("seed: {}  next token: '{}'".format(seed, next_token))

row_probs:
D3     0.333333
G3     0.666666
F#3    0.999999
Name: ['G#3', 'A3'], dtype: float64
seed: ['G#3', 'A3']  next token: 'G3'


In [12]:
# determine new seed
ns = seed.strip('[]').replace(" ","").replace("'","").split(',')
ns.append(next_token)
print(ns)
new_seed = str(ns[1:])
print(new_seed)

['A3', 'A#3', 'B3']
['A#3', 'B3']


## Durations chain

In [8]:
durpath = "/Compile/dwbzen/resources/music/bwv29_8_durationsChain.json"
dur_df = pd.read_json(durpath, orient="index")
dur_df

,1970-01-01 00:00:00.500,1970-01-01 00:00:02.000,1970-01-01 00:00:01.000
"[1.0, 1.0]",0.208,0.245,0.547
"[0.5, 0.5]",0.658,0.139,0.203
"[2.0, 1.0]",0.240,0.400,0.360
"[1.0, 0.5]",1.000,0.000,0.000
"[0.5, 2.0]",0.273,0.182,0.545
"[2.0, 2.0]",0.000,0.000,1.000
"[1.0, 2.0]",0.139,0.056,0.806
"[1.0, 1.5]",1.000,0.000,0.000
"[0.5, 1.0]",0.154,0.154,0.692
"[1.0, 3.0]",0.000,0.000,1.000


In [23]:
durationKeys =  pd.Series(dur_df.index)
durations_key_values = durationKeys.values
seed = "[2.0, 1.0]"
seed in durations_key_values

True

In [33]:
row = dur_df.loc[seed]
row_probs = row[row > 0].cumsum()
prob = random.random()
p = row_probs[row_probs> prob].iat[0]
next_token = row_probs[row_probs> prob].index[0]
next_token

Timestamp('1970-01-01 00:00:02')

In [32]:
next_token = 2.0

Timestamp('1970-01-01 00:00:01')